# Capstone Project: Data Collection
* Collecting Insider, House of Reps, Senate, and Congress data using Quiver quant alternative data source
* quiverquant rate limit: 100calls/second and 1000 calls per minute
* for each data set pull all tickers in SP500 and all recent trade tickers (if not in SP500)

In [1]:
import json
import requests
from pandas import json_normalize
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import math


f = open("/Users/austinpaxton/Documents/credentials/quiver_quant_cred.json")
data = json.load(f)

auth_token = data["auth_token"]
base_url = data["base_URL"]
print(base_url)

headers = {
    'accept': 'application/json',
    'X-CSRFToken': 'TyTJwjuEC7VV7mOqZ622haRaaUr0x0Ng4nrwSRFKQs7vdoBcJlK9qjAS69ghzhFu',
    'Authorization': f"Token {auth_token}"
}


https://api.quiverquant.com/beta/


In [2]:
#pull SP500 tickers
sp500_tickers = pd.read_csv("/Users/austinpaxton/Documents/ML Bootcamp/Capstone/SPY_companies.csv")
sp500_tickers = sp500_tickers.Symbol.tolist()
print(f"Length: {len(sp500_tickers)}")
#print(sp500_tickers)

Length: 502


In [3]:
# pull larger list of all tickers (5783 total) for more data
tickers_all = pd.read_csv("/Users/austinpaxton/Documents/ML Bootcamp/Capstone/stock_info.csv")
tickers_all = tickers_all.Ticker.unique().tolist()
print(len(tickers_all))
#looks like there were 3 duplicates removed w unique() method

5779


# Senate Trades

In [4]:
#get latest senate trades 
senate_url_recent = base_url + "live/senatetrading"
senate_data_recent = requests.get(senate_url_recent, headers=headers)


senate_trades=senate_data_recent.json()
print(type(senate_trades))
senate_trades = json_normalize(senate_trades)
print(senate_trades)
senate_tickers = senate_trades.Ticker.unique().tolist()
print(senate_tickers)

<class 'list'>
     Ticker        Date            Senator Transaction  Amount Party  \
0      QCOM  2023-02-28  Carper, Thomas R.        Sale  1001.0     D   
1       TSM  2023-02-21  Carper, Thomas R.        Sale  1001.0     D   
2       AGX  2023-02-21  Carper, Thomas R.    Purchase  1001.0     D   
3       KBE  2023-02-21  Carper, Thomas R.        Sale  1001.0     D   
4       FPI  2023-02-21  Carper, Thomas R.    Purchase  1001.0     D   
...     ...         ...                ...         ...     ...   ...   
3014     GD  2020-01-04      Boozman, John        Sale  1001.0     R   
3015   FBHS  2020-01-03      Boozman, John        Sale  1001.0     R   
3016   FISV  2020-01-02      Boozman, John        Sale  1001.0     R   
3017   ENTG  2020-01-02      Perdue, David        Sale  1001.0     R   
3018   ENTG  2020-01-02      Perdue, David        Sale  1001.0     R   

                 Range  
0     $1,001 - $15,000  
1     $1,001 - $15,000  
2     $1,001 - $15,000  
3     $1,001 - $15,0

In [37]:
# sanity check
"""since csv of all tickers was pulled from the internet we will see if all the recent trade tickers are incldued
in the tickers_all list"""

print(F" QTY of Tickers in tickers_all: {len(tickers_all)}")
tickers_all_merge = list(set(tickers_all+senate_tickers))
print(f"QTY of unqiue tickers after merging tickers_all and latest_trades: {len(tickers_all_merge)}")

print(len(set(tickers_all_merge)))

 QTY of Tickers in tickers_all: 5779
QTY of unqiue tickers after merging tickers_all and latest_trades: 6102
6102


In [53]:
# iteratively pull senate historical trades for tickers in senate_trades 
from jsonmerge import merge

#initialize variables
first_ticker = True
request_qty = 1000  #need to keep track of requests... rate limited to 1000 requests per minute
empty_response_tickers = []


for ticker in tickers_all_merge:
    if request_qty>0:  #only runs a request if there is requests available
        
        senate_url_temp = base_url + f"historical/senatetrading/{ticker}"
        senate_data_temp = requests.get(senate_url_temp, headers=headers).json()
        if request_qty ==1000:
            rate_lim_reset_time = datetime.now() + timedelta(minutes=1)  #gets time of first request so we know when to refresh the request qty

        print(f"--------Pulling Senate trades on Ticker: {ticker}------ \n")
        request_qty -= 1
            
            
        
        senate_data_temp = json_normalize(senate_data_temp) #convert data to json
        empty = senate_data_temp.empty  #cehck if data is empty
        
        #stores data if not empty and merges to existing pandas df
        if not empty:
            if first_ticker == True:
                senate_data_merge = senate_data_temp
                first_ticker = False
            else:
                senate_data_merge = senate_data_merge.merge(senate_data_temp,how="outer")
        else:
            empty_response_tickers.append(ticker)
                
    else:
        time_till_refresh = rate_lim_reset_time-datetime.now()
        wait_time = math.ceil(time_till_refresh.total_seconds())
        if wait_time<0:
            pass
        else: #wait for positive wait time
            print(f"------- Out of Requests sleeping for {wait_time}s ------")
            time.sleep(wait_time)
        request_qty = 1000  #reset request qunatity
        rate_lim_reset_time = datetime.now() + timedelta(minutes=1) #set new rate limit reset time
        

        
    


--------Pulling Senate trades on Ticker: HIS------ 

--------Pulling Senate trades on Ticker: DVM------ 

--------Pulling Senate trades on Ticker: CSG------ 

--------Pulling Senate trades on Ticker: NYT------ 

--------Pulling Senate trades on Ticker: MRH-A------ 

--------Pulling Senate trades on Ticker: FEI------ 

--------Pulling Senate trades on Ticker: ARRS------ 

--------Pulling Senate trades on Ticker: KW------ 

--------Pulling Senate trades on Ticker: NMO------ 

--------Pulling Senate trades on Ticker: MNTX------ 

--------Pulling Senate trades on Ticker: CSS------ 

--------Pulling Senate trades on Ticker: PUK------ 

--------Pulling Senate trades on Ticker: AWK------ 

--------Pulling Senate trades on Ticker: GL------ 

--------Pulling Senate trades on Ticker: APTV------ 

--------Pulling Senate trades on Ticker: RRC------ 

--------Pulling Senate trades on Ticker: FRBK------ 

--------Pulling Senate trades on Ticker: RGLD------ 

--------Pulling Senate trades on Ticker: 

--------Pulling Senate trades on Ticker: JNY------ 

--------Pulling Senate trades on Ticker: NP------ 

--------Pulling Senate trades on Ticker: LIQD------ 

--------Pulling Senate trades on Ticker: AWC------ 

--------Pulling Senate trades on Ticker: CARZ------ 

--------Pulling Senate trades on Ticker: DGAS------ 

--------Pulling Senate trades on Ticker: HMNF------ 

--------Pulling Senate trades on Ticker: ZIXI------ 

--------Pulling Senate trades on Ticker: ENL------ 

--------Pulling Senate trades on Ticker: LDF------ 

--------Pulling Senate trades on Ticker: IART------ 

--------Pulling Senate trades on Ticker: WLBPZ------ 

--------Pulling Senate trades on Ticker: STI.B------ 

--------Pulling Senate trades on Ticker: EHI------ 

--------Pulling Senate trades on Ticker: BIOS------ 

--------Pulling Senate trades on Ticker: FRFHF------ 

--------Pulling Senate trades on Ticker: BLMT------ 

--------Pulling Senate trades on Ticker: OTEX------ 

--------Pulling Senate trades on

--------Pulling Senate trades on Ticker: DATA------ 

--------Pulling Senate trades on Ticker: PFG-B------ 

--------Pulling Senate trades on Ticker: SCBT------ 

--------Pulling Senate trades on Ticker: AAP------ 

--------Pulling Senate trades on Ticker: LMCA------ 

--------Pulling Senate trades on Ticker: EDUC------ 

--------Pulling Senate trades on Ticker: LUX------ 

--------Pulling Senate trades on Ticker: HIIQ------ 

--------Pulling Senate trades on Ticker: SWY------ 

--------Pulling Senate trades on Ticker: HTLF------ 

--------Pulling Senate trades on Ticker: DEG------ 

--------Pulling Senate trades on Ticker: ARH-C------ 

--------Pulling Senate trades on Ticker: CO------ 

--------Pulling Senate trades on Ticker: NAZ------ 

--------Pulling Senate trades on Ticker: DFRG------ 

--------Pulling Senate trades on Ticker: PTSX------ 

--------Pulling Senate trades on Ticker: RE------ 

--------Pulling Senate trades on Ticker: BRLI------ 

--------Pulling Senate trades on Ti

--------Pulling Senate trades on Ticker: GSMYX------ 

--------Pulling Senate trades on Ticker: KAP------ 

--------Pulling Senate trades on Ticker: ADAT------ 

--------Pulling Senate trades on Ticker: FHN-A------ 

--------Pulling Senate trades on Ticker: ABTL------ 

--------Pulling Senate trades on Ticker: MGRC------ 

--------Pulling Senate trades on Ticker: GFN------ 

--------Pulling Senate trades on Ticker: DDC------ 

--------Pulling Senate trades on Ticker: DD-B------ 

--------Pulling Senate trades on Ticker: CFG------ 

--------Pulling Senate trades on Ticker: ETB------ 

--------Pulling Senate trades on Ticker: CG------ 

--------Pulling Senate trades on Ticker: MYE------ 

--------Pulling Senate trades on Ticker: OFC-L------ 

--------Pulling Senate trades on Ticker: BEE-A------ 

--------Pulling Senate trades on Ticker: GGS------ 

--------Pulling Senate trades on Ticker: BECN------ 

--------Pulling Senate trades on Ticker: MILL------ 

--------Pulling Senate trades on 

--------Pulling Senate trades on Ticker: CPIX------ 

--------Pulling Senate trades on Ticker: XWES------ 

--------Pulling Senate trades on Ticker: XOMA------ 

--------Pulling Senate trades on Ticker: HOVNP------ 

--------Pulling Senate trades on Ticker: MPC------ 

--------Pulling Senate trades on Ticker: FDI------ 

--------Pulling Senate trades on Ticker: QLYS------ 

--------Pulling Senate trades on Ticker: OLED------ 

--------Pulling Senate trades on Ticker: LIVE------ 

--------Pulling Senate trades on Ticker: ESC------ 

--------Pulling Senate trades on Ticker: RDHL------ 

--------Pulling Senate trades on Ticker: INGR------ 

--------Pulling Senate trades on Ticker: MPG-A------ 

--------Pulling Senate trades on Ticker: VOO------ 

--------Pulling Senate trades on Ticker: NZH-C------ 

--------Pulling Senate trades on Ticker: LBTYA------ 

--------Pulling Senate trades on Ticker: TYN------ 

--------Pulling Senate trades on Ticker: AMCF------ 

--------Pulling Senate trades

...

--------Pulling Senate trades on Ticker: VICR------ 

--------Pulling Senate trades on Ticker: SGNT------ 

--------Pulling Senate trades on Ticker: CBEY------ 

--------Pulling Senate trades on Ticker: DAN------ 

--------Pulling Senate trades on Ticker: LVS------ 

--------Pulling Senate trades on Ticker: PHM------ 

--------Pulling Senate trades on Ticker: RCKB------ 

--------Pulling Senate trades on Ticker: UCBI------ 

--------Pulling Senate trades on Ticker: BGSCU------ 

--------Pulling Senate trades on Ticker: FGBI------ 

--------Pulling Senate trades on Ticker: EPAX------ 

--------Pulling Senate trades on Ticker: SQQQ------ 

--------Pulling Senate trades on Ticker: BTF------ 

--------Pulling Senate trades on Ticker: LF------ 

--------Pulling Senate trades on Ticker: BIE------ 

--------Pulling Senate trades on Ticker: DMD------ 

--------Pulling Senate trades on Ticker: GZT------ 

--------Pulling Senate trades on Ticker: OSBC------ 

--------Pulling Senate trades on Tic

--------Pulling Senate trades on Ticker: TE------ 

--------Pulling Senate trades on Ticker: HDGE------ 

--------Pulling Senate trades on Ticker: BK------ 

--------Pulling Senate trades on Ticker: KTCC------ 

--------Pulling Senate trades on Ticker: NXP------ 

--------Pulling Senate trades on Ticker: XRSC------ 

--------Pulling Senate trades on Ticker: FHCO------ 

--------Pulling Senate trades on Ticker: BIG------ 

--------Pulling Senate trades on Ticker: ECPG------ 

--------Pulling Senate trades on Ticker: RESI------ 

--------Pulling Senate trades on Ticker: PEBO------ 

--------Pulling Senate trades on Ticker: VLTR------ 

--------Pulling Senate trades on Ticker: BMA------ 

--------Pulling Senate trades on Ticker: GTXI------ 

--------Pulling Senate trades on Ticker: CEE------ 

--------Pulling Senate trades on Ticker: SO------ 

--------Pulling Senate trades on Ticker: GTN------ 

--------Pulling Senate trades on Ticker: SPAN------ 

--------Pulling Senate trades on Ticker

--------Pulling Senate trades on Ticker: RBA------ 

--------Pulling Senate trades on Ticker: FUNC------ 

--------Pulling Senate trades on Ticker: IDTI------ 

--------Pulling Senate trades on Ticker: ROMA------ 

--------Pulling Senate trades on Ticker: PRCP------ 

--------Pulling Senate trades on Ticker: PDLI------ 

--------Pulling Senate trades on Ticker: HWKN------ 

--------Pulling Senate trades on Ticker: AHT------ 

--------Pulling Senate trades on Ticker: EXP------ 

--------Pulling Senate trades on Ticker: WABC------ 

--------Pulling Senate trades on Ticker: VRSN------ 

--------Pulling Senate trades on Ticker: WHR------ 

--------Pulling Senate trades on Ticker: GE------ 

--------Pulling Senate trades on Ticker: HTA------ 

--------Pulling Senate trades on Ticker: MARPS------ 

--------Pulling Senate trades on Ticker: PBY------ 

--------Pulling Senate trades on Ticker: UPL------ 

--------Pulling Senate trades on Ticker: KOPN------ 

--------Pulling Senate trades on Tic

--------Pulling Senate trades on Ticker: EQT------ 

--------Pulling Senate trades on Ticker: ALG------ 

--------Pulling Senate trades on Ticker: MTL.P------ 

--------Pulling Senate trades on Ticker: IGR------ 

--------Pulling Senate trades on Ticker: UBNK------ 

--------Pulling Senate trades on Ticker: SPNS------ 

--------Pulling Senate trades on Ticker: POPE------ 

--------Pulling Senate trades on Ticker: SUPX------ 

--------Pulling Senate trades on Ticker: ALTR------ 

--------Pulling Senate trades on Ticker: KT------ 

--------Pulling Senate trades on Ticker: PRIM------ 

--------Pulling Senate trades on Ticker: MUE------ 

--------Pulling Senate trades on Ticker: CCG------ 

--------Pulling Senate trades on Ticker: PGEM------ 

--------Pulling Senate trades on Ticker: KMB------ 

--------Pulling Senate trades on Ticker: RTI------ 

--------Pulling Senate trades on Ticker: XRAY------ 

--------Pulling Senate trades on Ticker: IWN------ 

--------Pulling Senate trades on Tick

--------Pulling Senate trades on Ticker: WBCO------ 

--------Pulling Senate trades on Ticker: FULT------ 

--------Pulling Senate trades on Ticker: HAL------ 

--------Pulling Senate trades on Ticker: AFH------ 

--------Pulling Senate trades on Ticker: UVV------ 

--------Pulling Senate trades on Ticker: LAZ------ 

--------Pulling Senate trades on Ticker: LACO------ 

--------Pulling Senate trades on Ticker: FT------ 

--------Pulling Senate trades on Ticker: CPK------ 

--------Pulling Senate trades on Ticker: POM------ 

--------Pulling Senate trades on Ticker: ODP------ 

--------Pulling Senate trades on Ticker: BBEP------ 

--------Pulling Senate trades on Ticker: SLT------ 

--------Pulling Senate trades on Ticker: FGB------ 

--------Pulling Senate trades on Ticker: BRKR------ 

--------Pulling Senate trades on Ticker: XNPT------ 

--------Pulling Senate trades on Ticker: ZTR------ 

--------Pulling Senate trades on Ticker: ALLE------ 

--------Pulling Senate trades on Ticker:

In [55]:
print(senate_data_merge)
unique_tickers = senate_data_merge.Ticker.unique()
print(f"Unique Ticker Count: {len(unique_tickers)}")
print(empty_response_tickers)

            Date Ticker            Senator Transaction   Amount  \
0     2022-01-14    AWK  Tuberville, Tommy        Sale  15001.0   
1     2016-12-01    AWK  Collins, Susan M.    Purchase  15001.0   
2     2016-12-01    AWK  Collins, Susan M.    Purchase  15001.0   
3     2018-12-27   APTV   Inhofe, James M.        Sale  15001.0   
4     2018-11-16   APTV   Inhofe, James M.        Sale  15001.0   
...          ...    ...                ...         ...      ...   
6210  2021-09-29    BHP  Tuberville, Tommy        Sale   1001.0   
6211  2021-03-29    BHP  Tuberville, Tommy    Purchase   1001.0   
6212  2020-07-30    OSK         Wyden, Ron    Purchase  15001.0   
6213  2020-07-30    OSK         Wyden, Ron    Purchase  15001.0   
6214  2022-10-06   PAYC  Tuberville, Tommy    Purchase   1001.0   

                  Range  
0     $15,001 - $50,000  
1     $15,001 - $50,000  
2     $15,001 - $50,000  
3     $15,001 - $50,000  
4     $15,001 - $50,000  
...                 ...  
6210   $1,001

In [57]:
#convert senate data merge into a csv file
senate_data_merge.to_csv("/Users/austinpaxton/Documents/ML Bootcamp/Capstone/senate_trades_all.csv")


# House trades

In [10]:
#Get latest house trades
house_url_recent = base_url  + "live/housetrading"
house_data_recent = requests.get(house_url_recent, headers=headers)
house_trades = json_normalize(house_data_recent.json())
house_trades.head()


house_tickers = house_trades.Ticker.unique().tolist()
print(len(senate_tickers))

673


In [11]:
#merge long tickers list with recent house trades tickers to loop through
house_tickers_all_merge = list(set(tickers_all+house_tickers))

In [14]:
# iteratively pull senate historical trades for tickers in senate_trades 
from jsonmerge import merge

#initialize variables
first_ticker = True
request_qty = 1000  #need to keep track of requests... rate limited to 1000 requests per minute
house_empty_response_tickers = []



for ticker in house_tickers_all_merge:
    if request_qty>0:  #only runs a request if there is requests available
        
        house_url_temp = base_url + f"historical/housetrading/{ticker}"
        house_data_temp = requests.get(house_url_temp, headers=headers).json()
        if request_qty ==1000:
            rate_lim_reset_time = datetime.now() + timedelta(minutes=1)  #gets time of first request so we know when to refresh the request qty

        print(f"--------Pulling Senate trades on Ticker: {ticker}  || {1+house_tickers_all_merge.index(ticker)}/{len(house_tickers_all_merge)}------ \n")
        request_qty -= 1
            
            
        
        house_data_temp = json_normalize(house_data_temp) #convert data to json
        empty = house_data_temp.empty  #check if data is empty
        
        #stores data if not empty and merges to existing pandas df
        if not empty:
            if first_ticker == True:
                house_data_merge = house_data_temp
                first_ticker = False
            else:
                house_data_merge = house_data_merge.merge(house_data_temp,how="outer")
        else:
            house_empty_response_tickers.append(ticker)
                
    else:
        time_till_refresh = rate_lim_reset_time-datetime.now()
        wait_time = math.ceil(time_till_refresh.total_seconds())
        if wait_time<0:
            pass
        else: #wait for positive wait time
            print(f"------- Out of Requests sleeping for {wait_time}s ------")
            time.sleep(wait_time)
        request_qty = 1000  #reset request qunatity
        rate_lim_reset_time = datetime.now() + timedelta(minutes=1) #set new rate limit reset time
        

        

--------Pulling Senate trades on Ticker: COTY  || 1/6337------ 

--------Pulling Senate trades on Ticker: EBS  || 2/6337------ 

--------Pulling Senate trades on Ticker: CMCSA  || 3/6337------ 

--------Pulling Senate trades on Ticker: VALU  || 4/6337------ 

--------Pulling Senate trades on Ticker: QABSY  || 5/6337------ 

--------Pulling Senate trades on Ticker: AVHI  || 6/6337------ 

--------Pulling Senate trades on Ticker: MSM  || 7/6337------ 

--------Pulling Senate trades on Ticker: JAPAY  || 8/6337------ 

--------Pulling Senate trades on Ticker: UFI  || 9/6337------ 

--------Pulling Senate trades on Ticker: PRGN  || 10/6337------ 

--------Pulling Senate trades on Ticker: KAMN  || 11/6337------ 

--------Pulling Senate trades on Ticker: AEHR  || 12/6337------ 

--------Pulling Senate trades on Ticker: PSEM  || 13/6337------ 

--------Pulling Senate trades on Ticker: HTWO  || 14/6337------ 

--------Pulling Senate trades on Ticker: TRMK  || 15/6337------ 

--------Pulling Sen

--------Pulling Senate trades on Ticker: STBFY  || 126/6337------ 

--------Pulling Senate trades on Ticker: ABT  || 127/6337------ 

--------Pulling Senate trades on Ticker: ARTX  || 128/6337------ 

--------Pulling Senate trades on Ticker: HCLP  || 129/6337------ 

--------Pulling Senate trades on Ticker: ORI  || 130/6337------ 

--------Pulling Senate trades on Ticker: KNOP  || 131/6337------ 

--------Pulling Senate trades on Ticker: PEI  || 132/6337------ 

--------Pulling Senate trades on Ticker: XUE  || 133/6337------ 

--------Pulling Senate trades on Ticker: ALL  || 134/6337------ 

--------Pulling Senate trades on Ticker: SDLP  || 135/6337------ 

--------Pulling Senate trades on Ticker: MWR  || 136/6337------ 

--------Pulling Senate trades on Ticker: MAMS  || 137/6337------ 

--------Pulling Senate trades on Ticker: SAN-C  || 138/6337------ 

--------Pulling Senate trades on Ticker: TSM  || 139/6337------ 

--------Pulling Senate trades on Ticker: NPY  || 140/6337------ 

-

--------Pulling Senate trades on Ticker: JD  || 250/6337------ 

--------Pulling Senate trades on Ticker: RBS-S  || 251/6337------ 

--------Pulling Senate trades on Ticker: MCO  || 252/6337------ 

--------Pulling Senate trades on Ticker: BBY  || 253/6337------ 

--------Pulling Senate trades on Ticker: INTL  || 254/6337------ 

--------Pulling Senate trades on Ticker: N  || 255/6337------ 

--------Pulling Senate trades on Ticker: SXI  || 256/6337------ 

--------Pulling Senate trades on Ticker: GMETP  || 257/6337------ 

--------Pulling Senate trades on Ticker: RM  || 258/6337------ 

--------Pulling Senate trades on Ticker: DIN  || 259/6337------ 

--------Pulling Senate trades on Ticker: MNK  || 260/6337------ 

--------Pulling Senate trades on Ticker: DCUA  || 261/6337------ 

--------Pulling Senate trades on Ticker: SCCO  || 262/6337------ 

--------Pulling Senate trades on Ticker: PYCR  || 263/6337------ 

--------Pulling Senate trades on Ticker: FULL  || 264/6337------ 

-----

--------Pulling Senate trades on Ticker: GJR  || 373/6337------ 

--------Pulling Senate trades on Ticker: XLS  || 374/6337------ 

--------Pulling Senate trades on Ticker: NWN  || 375/6337------ 

--------Pulling Senate trades on Ticker: MNI  || 376/6337------ 

--------Pulling Senate trades on Ticker: OBAF  || 377/6337------ 

--------Pulling Senate trades on Ticker: VNR  || 378/6337------ 

--------Pulling Senate trades on Ticker: PAYX  || 379/6337------ 

--------Pulling Senate trades on Ticker: EMFN  || 380/6337------ 

--------Pulling Senate trades on Ticker: AGO-F  || 381/6337------ 

--------Pulling Senate trades on Ticker: FBR  || 382/6337------ 

--------Pulling Senate trades on Ticker: AMH  || 383/6337------ 

--------Pulling Senate trades on Ticker: FVRR  || 384/6337------ 

--------Pulling Senate trades on Ticker: WRK  || 385/6337------ 

--------Pulling Senate trades on Ticker: VEEV  || 386/6337------ 

--------Pulling Senate trades on Ticker: NLY-A  || 387/6337------ 

-

--------Pulling Senate trades on Ticker: JMI  || 496/6337------ 

--------Pulling Senate trades on Ticker: SHOS  || 497/6337------ 

--------Pulling Senate trades on Ticker: HTH  || 498/6337------ 

--------Pulling Senate trades on Ticker: BICK  || 499/6337------ 

--------Pulling Senate trades on Ticker: CPSI  || 500/6337------ 

--------Pulling Senate trades on Ticker: UNTD  || 501/6337------ 

--------Pulling Senate trades on Ticker: TPRE  || 502/6337------ 

--------Pulling Senate trades on Ticker: FRX  || 503/6337------ 

--------Pulling Senate trades on Ticker: ALCO  || 504/6337------ 

--------Pulling Senate trades on Ticker: VRA  || 505/6337------ 

--------Pulling Senate trades on Ticker: ROIAK  || 506/6337------ 

--------Pulling Senate trades on Ticker: HPCCP  || 507/6337------ 

--------Pulling Senate trades on Ticker: CAPL  || 508/6337------ 

--------Pulling Senate trades on Ticker: GH  || 509/6337------ 

--------Pulling Senate trades on Ticker: FFIC  || 510/6337------ 


...

--------Pulling Senate trades on Ticker: FPT  || 5760/6337------ 

--------Pulling Senate trades on Ticker: ALB  || 5761/6337------ 

--------Pulling Senate trades on Ticker: CRNC  || 5762/6337------ 

--------Pulling Senate trades on Ticker: MFG  || 5763/6337------ 

--------Pulling Senate trades on Ticker: CSU  || 5764/6337------ 

--------Pulling Senate trades on Ticker: GOV  || 5765/6337------ 

--------Pulling Senate trades on Ticker: MVO  || 5766/6337------ 

--------Pulling Senate trades on Ticker: YGE  || 5767/6337------ 

--------Pulling Senate trades on Ticker: POWR  || 5768/6337------ 

--------Pulling Senate trades on Ticker: JEF  || 5769/6337------ 

--------Pulling Senate trades on Ticker: ERIC  || 5770/6337------ 

--------Pulling Senate trades on Ticker: FCE.A  || 5771/6337------ 

--------Pulling Senate trades on Ticker: INFY  || 5772/6337------ 

--------Pulling Senate trades on Ticker: DAR  || 5773/6337------ 

--------Pulling Senate trades on Ticker: FRC-A  || 5774/

--------Pulling Senate trades on Ticker: OMC  || 5883/6337------ 

--------Pulling Senate trades on Ticker: CW  || 5884/6337------ 

--------Pulling Senate trades on Ticker: IDEXY  || 5885/6337------ 

--------Pulling Senate trades on Ticker: FLGB  || 5886/6337------ 

--------Pulling Senate trades on Ticker: BEE-B  || 5887/6337------ 

--------Pulling Senate trades on Ticker: NID  || 5888/6337------ 

--------Pulling Senate trades on Ticker: ZOLT  || 5889/6337------ 

--------Pulling Senate trades on Ticker: IMN  || 5890/6337------ 

--------Pulling Senate trades on Ticker: RMTI  || 5891/6337------ 

--------Pulling Senate trades on Ticker: VVNT  || 5892/6337------ 

--------Pulling Senate trades on Ticker: ROP  || 5893/6337------ 

--------Pulling Senate trades on Ticker: FCH-A  || 5894/6337------ 

--------Pulling Senate trades on Ticker: NVY-C  || 5895/6337------ 

--------Pulling Senate trades on Ticker: WSTC  || 5896/6337------ 

--------Pulling Senate trades on Ticker: LCUT  || 

--------Pulling Senate trades on Ticker: HAS  || 6005/6337------ 

--------Pulling Senate trades on Ticker: GBNK  || 6007/6337------ 

--------Pulling Senate trades on Ticker: JEPI  || 6008/6337------ 

--------Pulling Senate trades on Ticker: OZM  || 6009/6337------ 

--------Pulling Senate trades on Ticker: NVC  || 6010/6337------ 

--------Pulling Senate trades on Ticker: MPW  || 6011/6337------ 

--------Pulling Senate trades on Ticker: AHT  || 6012/6337------ 

--------Pulling Senate trades on Ticker: EIG  || 6013/6337------ 

--------Pulling Senate trades on Ticker: GCO  || 6014/6337------ 

--------Pulling Senate trades on Ticker: QTM  || 6015/6337------ 

--------Pulling Senate trades on Ticker: GIII  || 6016/6337------ 

--------Pulling Senate trades on Ticker: SNX  || 6017/6337------ 

--------Pulling Senate trades on Ticker: BCOV  || 6018/6337------ 

--------Pulling Senate trades on Ticker: HPI  || 6019/6337------ 

--------Pulling Senate trades on Ticker: ADAT  || 6020/633

--------Pulling Senate trades on Ticker: BANFP  || 6128/6337------ 

--------Pulling Senate trades on Ticker: SEM.W  || 6129/6337------ 

--------Pulling Senate trades on Ticker: PNFP  || 6130/6337------ 

--------Pulling Senate trades on Ticker: TRGP  || 6131/6337------ 

--------Pulling Senate trades on Ticker: XOM  || 6132/6337------ 

--------Pulling Senate trades on Ticker: SMTC  || 6133/6337------ 

--------Pulling Senate trades on Ticker: BLW  || 6134/6337------ 

--------Pulling Senate trades on Ticker: MS$P  || 6135/6337------ 

--------Pulling Senate trades on Ticker: VBF  || 6136/6337------ 

--------Pulling Senate trades on Ticker: KCG  || 6137/6337------ 

--------Pulling Senate trades on Ticker: DGICB  || 6138/6337------ 

--------Pulling Senate trades on Ticker: BLDP  || 6139/6337------ 

--------Pulling Senate trades on Ticker: PDEX  || 6140/6337------ 

--------Pulling Senate trades on Ticker: AIV  || 6141/6337------ 

--------Pulling Senate trades on Ticker: ALV  || 6

--------Pulling Senate trades on Ticker: ANDAU  || 6251/6337------ 

--------Pulling Senate trades on Ticker: LITB  || 6252/6337------ 

--------Pulling Senate trades on Ticker: QIHU  || 6253/6337------ 

--------Pulling Senate trades on Ticker: BKEPP  || 6254/6337------ 

--------Pulling Senate trades on Ticker: CVX  || 6255/6337------ 

--------Pulling Senate trades on Ticker: SBNYW  || 6256/6337------ 

--------Pulling Senate trades on Ticker: AMRC  || 6257/6337------ 

--------Pulling Senate trades on Ticker: GEO  || 6258/6337------ 

--------Pulling Senate trades on Ticker: GGE  || 6259/6337------ 

--------Pulling Senate trades on Ticker: MLNX  || 6260/6337------ 

--------Pulling Senate trades on Ticker: HOTR  || 6261/6337------ 

--------Pulling Senate trades on Ticker: TRF  || 6262/6337------ 

--------Pulling Senate trades on Ticker: WETF  || 6263/6337------ 

--------Pulling Senate trades on Ticker: AHPI  || 6264/6337------ 

--------Pulling Senate trades on Ticker: RALY  ||

In [17]:
house_data_merge.head()
house_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19065 entries, 0 to 19064
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            19065 non-null  object 
 1   Ticker          19065 non-null  object 
 2   Representative  19065 non-null  object 
 3   Transaction     19065 non-null  object 
 4   Amount          19065 non-null  float64
 5   Range           19065 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.0+ MB


In [18]:
#convert house data merge into a csv file
house_data_merge.to_csv("/Users/austinpaxton/Documents/ML Bootcamp/Capstone/house_trades_all.csv")


# Congress Trades


In [22]:
# get latest trades (all tickers)
congress_url_recent = base_url + "live/congresstrading"
congress_data_recent = requests.get(congress_url_recent, headers=headers)

print(congress_data_recent)
congress_trades = json_normalize(congress_data_recent.json())
congress_tickers = congress_trades.Ticker.unique().tolist()
congress_tickers_all = list(set(tickers_all+congress_tickers))
print(len(congress_tickers_all))

<Response [200]>
5990


In [23]:
# iteratively pull congress historical trades for tickers in senate_trades 
from jsonmerge import merge

#initialize variables
first_ticker = True
request_qty = 1000  #need to keep track of requests... rate limited to 1000 requests per minute
congress_empty_response_tickers = []



for ticker in congress_tickers_all:
    if request_qty>0:  #only runs a request if there is requests available
        
        congress_url_temp = base_url + f"historical/congresstrading/{ticker}"
        congress_data_temp = requests.get(congress_url_temp, headers=headers).json()
        if request_qty ==1000:
            rate_lim_reset_time = datetime.now() + timedelta(minutes=1)  #gets time of first request so we know when to refresh the request qty

        print(f"--------Pulling Congress trades on Ticker: {ticker}  || {1+congress_tickers_all.index(ticker)}/{len(congress_tickers_all)}------ \n")
        request_qty -= 1
            
            
        
        congress_data_temp = json_normalize(congress_data_temp) #convert data to json
        empty =congress_data_temp.empty  #check if data is empty
        
        #stores data if not empty and merges to existing pandas df
        if not empty:
            if first_ticker == True:
                congress_data_merge = congress_data_temp
                first_ticker = False
            else:
                congress_data_merge = congress_data_merge.merge(congress_data_temp,how="outer")
        else:
            congress_empty_response_tickers.append(ticker)
                
    else:
        time_till_refresh = rate_lim_reset_time-datetime.now()
        wait_time = math.ceil(time_till_refresh.total_seconds())
        if wait_time<0:
            pass
        else: #wait for positive wait time
            print(f"------- Out of Requests sleeping for {wait_time}s ------")
            time.sleep(wait_time)
        request_qty = 1000  #reset request qunatity
        rate_lim_reset_time = datetime.now() + timedelta(minutes=1) #set new rate limit reset time
        


--------Pulling Congress trades on Ticker: COTY  || 1/5990------ 

--------Pulling Congress trades on Ticker: EBS  || 2/5990------ 

--------Pulling Congress trades on Ticker: CMCSA  || 3/5990------ 

--------Pulling Congress trades on Ticker: VALU  || 4/5990------ 

--------Pulling Congress trades on Ticker: QABSY  || 5/5990------ 

--------Pulling Congress trades on Ticker: AVHI  || 6/5990------ 

--------Pulling Congress trades on Ticker: MSM  || 7/5990------ 

--------Pulling Congress trades on Ticker: JAPAY  || 8/5990------ 

--------Pulling Congress trades on Ticker: UFI  || 9/5990------ 

--------Pulling Congress trades on Ticker: PRGN  || 10/5990------ 

--------Pulling Congress trades on Ticker: KAMN  || 11/5990------ 

--------Pulling Congress trades on Ticker: AEHR  || 12/5990------ 

--------Pulling Congress trades on Ticker: PSEM  || 13/5990------ 

--------Pulling Congress trades on Ticker: HTWO  || 14/5990------ 

--------Pulling Congress trades on Ticker: TRMK  || 15/59

--------Pulling Congress trades on Ticker: ORI  || 123/5990------ 

--------Pulling Congress trades on Ticker: KNOP  || 124/5990------ 

--------Pulling Congress trades on Ticker: PEI  || 125/5990------ 

--------Pulling Congress trades on Ticker: XUE  || 126/5990------ 

--------Pulling Congress trades on Ticker: ALL  || 127/5990------ 

--------Pulling Congress trades on Ticker: SDLP  || 128/5990------ 

--------Pulling Congress trades on Ticker: MWR  || 129/5990------ 

--------Pulling Congress trades on Ticker: MAMS  || 130/5990------ 

--------Pulling Congress trades on Ticker: SAN-C  || 131/5990------ 

--------Pulling Congress trades on Ticker: TSM  || 132/5990------ 

--------Pulling Congress trades on Ticker: NPY  || 133/5990------ 

--------Pulling Congress trades on Ticker: ODC  || 134/5990------ 

--------Pulling Congress trades on Ticker: AAT  || 135/5990------ 

--------Pulling Congress trades on Ticker: LABL  || 136/5990------ 

--------Pulling Congress trades on Ticker:

--------Pulling Congress trades on Ticker: DIN  || 244/5990------ 

--------Pulling Congress trades on Ticker: MNK  || 245/5990------ 

--------Pulling Congress trades on Ticker: DCUA  || 246/5990------ 

--------Pulling Congress trades on Ticker: SCCO  || 247/5990------ 

--------Pulling Congress trades on Ticker: FULL  || 248/5990------ 

--------Pulling Congress trades on Ticker: OIIM  || 249/5990------ 

--------Pulling Congress trades on Ticker: GENE  || 250/5990------ 

--------Pulling Congress trades on Ticker: NMK-C  || 251/5990------ 

--------Pulling Congress trades on Ticker: ELUXY  || 252/5990------ 

--------Pulling Congress trades on Ticker: SIMG  || 253/5990------ 

--------Pulling Congress trades on Ticker: JCP  || 254/5990------ 

--------Pulling Congress trades on Ticker: NKG-C  || 255/5990------ 

--------Pulling Congress trades on Ticker: MDCA  || 256/5990------ 

--------Pulling Congress trades on Ticker: WAVX  || 257/5990------ 

--------Pulling Congress trades on

--------Pulling Congress trades on Ticker: UNTK  || 364/5990------ 

--------Pulling Congress trades on Ticker: MSON  || 365/5990------ 

--------Pulling Congress trades on Ticker: PRE-D  || 366/5990------ 

--------Pulling Congress trades on Ticker: GEF  || 367/5990------ 

--------Pulling Congress trades on Ticker: AG  || 368/5990------ 

--------Pulling Congress trades on Ticker: VICL  || 369/5990------ 

--------Pulling Congress trades on Ticker: CSCD  || 370/5990------ 

--------Pulling Congress trades on Ticker: DLB  || 371/5990------ 

--------Pulling Congress trades on Ticker: PYPL  || 372/5990------ 

--------Pulling Congress trades on Ticker: ARIA  || 373/5990------ 

--------Pulling Congress trades on Ticker: TAX  || 374/5990------ 

--------Pulling Congress trades on Ticker: ESLOY  || 375/5990------ 

--------Pulling Congress trades on Ticker: NEA-C  || 376/5990------ 

--------Pulling Congress trades on Ticker: GRO  || 377/5990------ 

--------Pulling Congress trades on Ti

--------Pulling Congress trades on Ticker: HJR  || 484/5990------ 

--------Pulling Congress trades on Ticker: FCN  || 485/5990------ 

--------Pulling Congress trades on Ticker: HALO  || 486/5990------ 

--------Pulling Congress trades on Ticker: ICGE  || 487/5990------ 

--------Pulling Congress trades on Ticker: PII  || 488/5990------ 

--------Pulling Congress trades on Ticker: TILE  || 489/5990------ 

--------Pulling Congress trades on Ticker: GSJ  || 490/5990------ 

--------Pulling Congress trades on Ticker: KAR  || 491/5990------ 

--------Pulling Congress trades on Ticker: ASGN  || 492/5990------ 

--------Pulling Congress trades on Ticker: MEOH  || 493/5990------ 

--------Pulling Congress trades on Ticker: CHN  || 494/5990------ 

--------Pulling Congress trades on Ticker: MCHP  || 495/5990------ 

--------Pulling Congress trades on Ticker: CAC  || 496/5990------ 

--------Pulling Congress trades on Ticker: SYNM  || 497/5990------ 

--------Pulling Congress trades on Ticker

...

--------Pulling Congress trades on Ticker: BXMT  || 5466/5990------ 

--------Pulling Congress trades on Ticker: GMT  || 5467/5990------ 

--------Pulling Congress trades on Ticker: HLN  || 5468/5990------ 

--------Pulling Congress trades on Ticker: CME  || 5469/5990------ 

--------Pulling Congress trades on Ticker: ACRX  || 5470/5990------ 

--------Pulling Congress trades on Ticker: DOC  || 5471/5990------ 

--------Pulling Congress trades on Ticker: BCO  || 5472/5990------ 

--------Pulling Congress trades on Ticker: NMY-C  || 5473/5990------ 

--------Pulling Congress trades on Ticker: EMN  || 5474/5990------ 

--------Pulling Congress trades on Ticker: DRU  || 5475/5990------ 

--------Pulling Congress trades on Ticker: OII  || 5476/5990------ 

--------Pulling Congress trades on Ticker: NUC  || 5477/5990------ 

--------Pulling Congress trades on Ticker: CNSI  || 5478/5990------ 

--------Pulling Congress trades on Ticker: BZT  || 5479/5990------ 

--------Pulling Congress trad

--------Pulling Congress trades on Ticker: LXFR  || 5584/5990------ 

--------Pulling Congress trades on Ticker: CMRX  || 5585/5990------ 

--------Pulling Congress trades on Ticker: UBP-D  || 5586/5990------ 

--------Pulling Congress trades on Ticker: TK  || 5587/5990------ 

--------Pulling Congress trades on Ticker: IBCP  || 5588/5990------ 

--------Pulling Congress trades on Ticker: NFEC  || 5589/5990------ 

--------Pulling Congress trades on Ticker: OWW  || 5590/5990------ 

--------Pulling Congress trades on Ticker: NDSN  || 5591/5990------ 

--------Pulling Congress trades on Ticker: PFSI  || 5592/5990------ 

--------Pulling Congress trades on Ticker: SFST  || 5593/5990------ 

--------Pulling Congress trades on Ticker: CAVM  || 5594/5990------ 

--------Pulling Congress trades on Ticker: ULTI  || 5595/5990------ 

--------Pulling Congress trades on Ticker: WSBF  || 5596/5990------ 

--------Pulling Congress trades on Ticker: SBRAP  || 5597/5990------ 

--------Pulling Congr

--------Pulling Congress trades on Ticker: CLACU  || 5702/5990------ 

--------Pulling Congress trades on Ticker: SBSI  || 5703/5990------ 

--------Pulling Congress trades on Ticker: SFG  || 5704/5990------ 

--------Pulling Congress trades on Ticker: TAOM  || 5705/5990------ 

--------Pulling Congress trades on Ticker: CXPO  || 5706/5990------ 

--------Pulling Congress trades on Ticker: EHI  || 5707/5990------ 

--------Pulling Congress trades on Ticker: SOHU  || 5708/5990------ 

--------Pulling Congress trades on Ticker: PAY  || 5709/5990------ 

--------Pulling Congress trades on Ticker: STAN  || 5710/5990------ 

--------Pulling Congress trades on Ticker: LMNX  || 5711/5990------ 

--------Pulling Congress trades on Ticker: GLF  || 5712/5990------ 

--------Pulling Congress trades on Ticker: MERU  || 5713/5990------ 

--------Pulling Congress trades on Ticker: NTC  || 5714/5990------ 

--------Pulling Congress trades on Ticker: GBX  || 5715/5990------ 

--------Pulling Congress 

--------Pulling Congress trades on Ticker: RDNT  || 5820/5990------ 

--------Pulling Congress trades on Ticker: FFN  || 5821/5990------ 

--------Pulling Congress trades on Ticker: EPD  || 5822/5990------ 

--------Pulling Congress trades on Ticker: HIW  || 5823/5990------ 

--------Pulling Congress trades on Ticker: WMGI  || 5824/5990------ 

--------Pulling Congress trades on Ticker: FSTR  || 5825/5990------ 

--------Pulling Congress trades on Ticker: AGC  || 5826/5990------ 

--------Pulling Congress trades on Ticker: ARL  || 5827/5990------ 

--------Pulling Congress trades on Ticker: ARMH  || 5828/5990------ 

--------Pulling Congress trades on Ticker: SWKS  || 5829/5990------ 

--------Pulling Congress trades on Ticker: GOLD  || 5830/5990------ 

--------Pulling Congress trades on Ticker: SPR  || 5831/5990------ 

--------Pulling Congress trades on Ticker: CUK  || 5832/5990------ 

--------Pulling Congress trades on Ticker: AMWD  || 5833/5990------ 

--------Pulling Congress tr

--------Pulling Congress trades on Ticker: PJH  || 5938/5990------ 

--------Pulling Congress trades on Ticker: ABNB  || 5939/5990------ 

--------Pulling Congress trades on Ticker: NTRI  || 5940/5990------ 

--------Pulling Congress trades on Ticker: RESI  || 5941/5990------ 

--------Pulling Congress trades on Ticker: PCYC  || 5942/5990------ 

--------Pulling Congress trades on Ticker: SUPN  || 5943/5990------ 

--------Pulling Congress trades on Ticker: OUTR  || 5944/5990------ 

--------Pulling Congress trades on Ticker: PAI  || 5945/5990------ 

--------Pulling Congress trades on Ticker: SYNT  || 5946/5990------ 

--------Pulling Congress trades on Ticker: LGP  || 5947/5990------ 

--------Pulling Congress trades on Ticker: GDV  || 5948/5990------ 

--------Pulling Congress trades on Ticker: RKT  || 5949/5990------ 

--------Pulling Congress trades on Ticker: FFNM  || 5950/5990------ 

--------Pulling Congress trades on Ticker: CREG  || 5951/5990------ 

--------Pulling Congress 

In [25]:
congress_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22620 entries, 0 to 22619
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ReportDate       22261 non-null  object 
 1   TransactionDate  22620 non-null  object 
 2   Ticker           22620 non-null  object 
 3   Representative   22620 non-null  object 
 4   Transaction      22620 non-null  object 
 5   Amount           22620 non-null  float64
 6   House            22620 non-null  object 
 7   Range            22620 non-null  object 
dtypes: float64(1), object(7)
memory usage: 1.6+ MB


In [26]:
congress_data_merge.to_csv("/Users/austinpaxton/Documents/ML Bootcamp/Capstone/congress_trades_all.csv")

# Insider trades

In [4]:
# returns recent insider transactions
insider_url = base_url + "live/insiders"
insider_data = requests.get(insider_url, headers=headers)

print(insider_data)

insider_df = json_normalize(insider_data.json())
insider_df.info()
insider_df.head()

<Response [200]>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Ticker                19949 non-null  object 
 1   Date                  20000 non-null  object 
 2   Name                  20000 non-null  object 
 3   AcquiredDisposedCode  20000 non-null  object 
 4   TransactionCode       20000 non-null  object 
 5   Shares                20000 non-null  float64
 6   PricePerShare         18508 non-null  float64
 7   SharesOwnedFollowing  19995 non-null  float64
 8   fileDate              20000 non-null  object 
dtypes: float64(3), object(6)
memory usage: 1.4+ MB


,Ticker,Date,Name,AcquiredDisposedCode,TransactionCode,Shares,PricePerShare,SharesOwnedFollowing,fileDate
0,PWSC,2023-03-25T00:00:00.000Z,GULATI HARDEEP,A,A,128337.0,0.00,2443293.0,2023-03-29T01:59:29.000Z
1,PWSC,2023-03-25T00:00:00.000Z,Shander Eric Ryan,A,A,89836.0,0.00,585907.0,2023-03-29T01:56:58.000Z
2,CNOB,2023-03-27T00:00:00.000Z,Magennis Elizabeth,D,F,1144.0,18.00,118673.0,2023-03-29T01:55:53.000Z
3,CNOB,2023-03-23T00:00:00.000Z,Magennis Elizabeth,D,F,1115.0,17.35,119817.0,2023-03-29T01:55:53.000Z
4,CNOB,2023-03-20T00:00:00.000Z,Magennis Elizabeth,D,F,2925.0,17.93,120932.0,2023-03-29T01:55:53.000Z


In [5]:
insider_df.to_csv("/Users/austinpaxton/Documents/ML Bootcamp/Capstone/insider_trades_03-27-23.csv")